In [1]:
using Pkg
Pkg.add("ProgressBars")

   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [2]:
using Random, LinearAlgebra, Plots
# bring packages into main namespace
using DataFrames             # Data tables are called "DataFrames"
using StatsPlots             # load plotting packages 
using Statistics             # basic statistical functions
using CSV                    # tools for working with CSV files
using Plots, Random, LinearAlgebra, Statistics, SparseArrays
using ProgressBars
include("proxgrad.jl")
pyplot()

Plots.PyPlotBackend()

In [41]:
dataset = CSV.read("Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets.csv")
dataset = dropmissing(dataset)
dataset[:when] = dataset[:Year] .+ ((dataset[:quarter] .-1) ./ 4)

n = size(dataset,1)

# For Extrapolation
cuttoff = 2015
before_idxs = dataset[:when].< cuttoff
after_idxs = dataset[:when].>= cuttoff

before_dataset = dataset[before_idxs,:]
after_dataset  = dataset[after_idxs,:]
    
n_before = size(before_dataset,1)
n_after = size(after_dataset,1)
@assert  n_before + n_after == n
@show n_before, n_after

head(dataset)

# For Interpolation
r = n_after / n
@show r
cuttoff = Int(round(n*r))
idxs = rand(1:n,n)

test_dataset  = dataset[idxs[1:cuttoff],:]
train_dataset = dataset[idxs[cuttoff+1:n],:]

n_train = size(train_dataset,1)
n_test = size(test_dataset,1)
@assert  n_train + n_test == n
@show n_train, n_test;

(n_before, n_after) = (170712, 30680)
r = 0.15233971557956622
(n_train, n_test) = (170712, 30680)


In [4]:
# Do k-fold cross validation and return the average error_metric on the validation set accross the k folds.
function cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(dataset,1)
    r = Int(round(n / k))
    idxs = rand(1:n,n) # to shuffle the dataset
    error = 0
    for i in (1:k)
        val_idxs = r*(i-1)+1:min(r*i,n)
        tr_low = 1:r*(i-1)
        tr_high = r*i+1:n
        if (i == 1)
            tr_idxs = tr_high
        elseif (i == k)
            tr_idxs = tr_low
        else
            tr_idxs = [tr_low ; tr_high ]
        end
        # @show i, val_idxs
        # @show tr_low, tr_high
        X_tr = X[idxs[tr_idxs],:]
        y_tr = y[idxs[tr_idxs]]
        
        X_val = X[idxs[val_idxs],:]
        y_val = y[idxs[val_idxs]]
        
        w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_val = X_val * w
        # @show size(y_val)
        # @show size(ŷ_val)
        error += error_metric(ŷ_val, y_val)
    end
    return error / k
end

function extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=10, dataset=train_dataset)
    X,y = featurizer(dataset)
    n = size(X,1)
    
    tr_idxs = dataset[:when].< 2013.5
    val_idxs = dataset[:when].>= 2013.5
    
    X_tr = X[tr_idxs,:]
    y_tr = y[tr_idxs]
    n_tr = size(X_tr,1)

    X_val = X[val_idxs,:]
    y_val = y[val_idxs]
    n_val = size(X_val,1)
    
    @assert n_tr + n_val == n

    w = proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
    ŷ_val = X_val * w
    # @show size(y_val)
    # @show size(ŷ_val)
    
    return error_metric(ŷ_val, y_val)
end

function train_and_validate(featurizer, loss, regularizer, stepsize, error_metric, mode; k=10, dataset=train_dataset)
    if (mode=="interpolation")
        return cross_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    else
        return extrapolation_val(featurizer, loss, regularizer, stepsize, error_metric; k=k, dataset=dataset)
    end
end

# For each model in models, do k-fold cross validation and calculate the average error_metric
# on the val set accross the k-folds.
# Each model in model is a tuple of the form (featurizer, loss, regularizer, stepsize),
# where a featurizer is a funciton that takes in a dataset and returns X,y. 
# Returns errors for each model, and the index of the best model
function test_models(models, error_metric,mode;k=10, dataset=train_dataset)
    errors = []
    for model in models
        error = train_and_validate(model...,error_metric,mode;k=k,dataset=dataset)
        errors = [errors; error]
    end
    i = argmin(errors)
    println("The best model is model ",i)
    return errors,i
end

test_models (generic function with 1 method)

# Feature Engineering and Selection

In [5]:
cat_labels = [:carrier_lg, :carrier_low, :airportid_1, :airportid_2]
cat_labels_sub = [:carrier_lg, :carrier_low]

#Sets of all categories in a particular column
cats_sets = [unique(dataset[:, label]) for label in cat_labels]
cats_sets_sub = [unique(dataset[:, label]) for label in cat_labels_sub]

"Computes a onehot vector for every entry in column given a set of categories cats"
function onehot(column, cats=unique(column))
    result = zeros(length(column), length(cats))
    for i = 1:length(column)
        for j =1:length(cats)
            if column[i] === cats[j]
                result[i, j] = 1
            end    
        end
    end
    result
end

onehot

In [6]:
labels_real = [
    :when, 
    :citymarketid_1, 
    :citymarketid_2,
    :airportid_1,
    :airportid_2,
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]
labels_real_sub = [
    :when, 
    :nsmiles,
    :passengers,
    :large_ms,
    :fare_lg,
    :lf_ms,
    :fare_low
]

7-element Array{Symbol,1}:
 :when
 :nsmiles
 :passengers
 :large_ms
 :fare_lg
 :lf_ms
 :fare_low

In [7]:
# Baseline featurizer
function feats_0(dataset)
    X = [dataset[:when] dataset[:airportid_1] dataset[:airportid_2] ones(size(dataset,1))]
    y = dataset[:fare]
    return X,y
end

# Only numeric features (treating airportid and cityid as numeric)
function feats_1(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    X = hcat(real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as numerical)
function feats_2(dataset)
    real_vals = convert(Matrix, dataset[labels_real])
    cat_vals = hcat([onehot(dataset[:, cat_labels_sub[i]], cats_sets_sub[i]) for i in 1:size(cat_labels_sub, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

# Numeric and categorical combined (treating airportid and cityid as categorical)
function feats_3(dataset)
    real_vals = convert(Matrix, dataset[labels_real_sub])
    cat_vals = hcat([onehot(dataset[:, cat_labels[i]], cats_sets[i]) for i in 1:size(cat_labels, 1)]...)
    X = hcat(cat_vals, real_vals, ones(size(dataset,1)))
    y = dataset[:fare]
    return X,y
end

MSE(L1,L2) = sum((L1.-L2).^2) / size(L1,1)

MSE (generic function with 1 method)

In [47]:
I0 = (feats_0, 1/n_train*QuadLoss(), 0.75*OneReg(), .1)
I1 = (feats_1, 1/n_train*QuadLoss(), ZeroReg(), .7)
I2 = (feats_2, 1/n_train*QuadLoss(), 0.5 *QuadReg(), .3)
I3 = (feats_3, 1/n_train*QuadLoss(), 0.75*QuadReg(), .7)

function report_results(models)
    results = []
    for model in tqdm(models)
        featurizer, loss, regularizer, stepsize = model

        # Interpolation
        X_tr, y_tr = featurizer(train_dataset)
        X_te, y_te = featurizer(test_dataset)
        w= proxgrad(loss, regularizer, X_tr, y_tr; stepsize=stepsize) 
        ŷ_te = X_te * w
        int_error = MSE(ŷ_te, y_te)

        # Extrapolation
        X_b4, y_b4 = featurizer(before_dataset)
        X_aftr, y_aftr = featurizer(after_dataset)
        w= proxgrad(loss, regularizer, X_b4, y_b4; stepsize=stepsize) 
        ŷ_aftr = X_aftr * w
        ext_error = MSE(ŷ_aftr, y_aftr)  

        results = [results; (int_error, ext_error) ]
    end
    return results
end
resultsI = report_results([I0, I1, I2, I3])

100.0%┣████████████████████████████████████████████┫ 4/4 [01:02<00:00, 0.0 it/s]


4-element Array{Any,1}:
 (6650.171657486256, 6209.362706969636)
 (6387.175258070116, 5990.595796144548)
 (6402.933123638795, 6006.5760827953045)
 (2781.3832035865066, 2398.174542660822)

In [48]:
E0 = (feats_0, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .7)
E1 = (feats_1, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1)
E2 = (feats_2, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .1)
E3 = (feats_3, 1/n_train*QuantileLoss(quantile=0.75), ZeroReg(), .3)
resultsE = report_results([E0, E1, E2, E3])

100.0%┣████████████████████████████████████████████┫ 4/4 [00:57<00:00, 0.1 it/s]


4-element Array{Any,1}:
 (8808.229596595665, 5375.917154490394)
 (8307.123462812846, 5113.792940533237)
 (8307.123452218753, 5113.792923855142)
 (4351.870356028882, 1965.275182648292)

In [51]:
@show resultsI
@show resultsE

for r in resultsI
    println(r[1], " & ", r[2], " \\\\")
end
println("\n")
for r in resultsE
    println(r[1], " & ", r[2], " \\\\")
end

resultsI = Any[(6650.171657486256, 6209.362706969636), (6387.175258070116, 5990.595796144548), (6402.933123638795, 6006.5760827953045), (2781.3832035865066, 2398.174542660822)]
resultsE = Any[(8808.229596595665, 5375.917154490394), (8307.123462812846, 5113.792940533237), (8307.123452218753, 5113.792923855142), (4351.870356028882, 1965.275182648292)]
6650.171657486256 & 6209.362706969636 \\
6387.175258070116 & 5990.595796144548 \\
6402.933123638795 & 6006.5760827953045 \\
2781.3832035865066 & 2398.174542660822 \\


8808.229596595665 & 5375.917154490394 \\
8307.123462812846 & 5113.792940533237 \\
8307.123452218753 & 5113.792923855142 \\
4351.870356028882 & 1965.275182648292 \\
